In [1]:
from main import *
from sklearn import clone

DROP_COLS = ["car_ownership", "house_ownership", "married"]

In [ ]:
transforms = [
    (correct_names, {"cols": ["profession", "city", "state"]}),
    (scaled_income, {"col": "city", "name": "income_by_city"}),
    (scaled_income, {"col": "state", "name": "income_by_state"}),
    (scaled_income, {"col": "profession", "name": "income_by_prof"}),
    (scale_cols, {"col": "income", "scale_col": "current_job_years", "name": "income_by_jobyrs"}),
    (scale_cols, {"col": "income", "scale_col": "age", "name": "income_by_age"}),
    (scale_cols, {"col": "income", "scale_col": "experience", "name": "income_by_exp"}),
    (scale_cols, {"col": "age", "scale_col": "experience", "name": "age_by_experience"}),
    (add_house_status, {}),
    (add_job_status, {})
]

df_train, df_test = data_preprocess([], transforms)
df_train.head(10)

In [ ]:
col = "current_job_years"
sns.histplot(x=col, data=df_train, label="train", color='g', stat="density", binwidth=1, alpha=0.5)
sns.histplot(x=col, data=df_test, label="test", color='r', stat="density", binwidth=1, alpha=0.5)
plt.show()

In [ ]:
model = RandomForestClassifier(n_jobs=-1, class_weight="balanced", max_features=1)
model = train_model(model, df_train)
features = df_train.drop("risk_flag", axis=1).columns
importances = model.feature_importances_
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
clf = RandomForestClassifier(n_jobs=-1, class_weight="balanced", max_features=1)
models = {
    "RandomForestV1": [clone(clf), ["car_ownership", "house_ownership", "married", "profession", "city", "state"]],
    "RandomForestAll": [clone(clf), []],
    "RandomForestV2": [clone(clf), ["profession", "city", "state"]],
    "RandomForestV3": [clone(clf), ["car_ownership", "house_ownership", "married"]]
}

res = pd.DataFrame.from_dict(
        {model_name: eval_model(model, data_preprocess(drop_cols, transforms)[0]) for model_name, (model, drop_cols) in models.items()}    
    )
sns.boxplot(data=res)

In [ ]:
model = RandomForestClassifier(n_estimators=1000, max_features=1, class_weight="balanced", n_jobs=-1)
df_train, df_test = data_preprocess(drop_cols=["car_ownership", "house_ownership", "married"], transform_fns=transforms)
sub = make_submission(model, df_train, df_test)
sub.to_csv("./tuning_rf/sub4.csv")

## Note: can safely remove `["car_ownership", "house_ownership", "married"]`

In [ ]:
drop_cols = ["car_ownership", "house_ownership", "married"]
df_train = data_preprocess(drop_cols, [transformfn1])[0]

param = "max_features"
par_vals = [i+1 for i in range(0, df_train.shape[1]-1, 2)]

res = pd.DataFrame.from_dict(
        {f"{param}_{val}": eval_model(RandomForestClassifier(n_jobs=-1, **{param: val}), df_train)
                                      for val in par_vals}
    )
sns.boxplot(data=res)

## Note: a smaller `max_features` value seems better
---

In [ ]:
drop_cols = ["car_ownership", "house_ownership", "married"]
df_train = data_preprocess(drop_cols, [income_by_state])[0]

param = "colsample_bytree"
par_vals = [0.6, 0.7, 0.8, 0.9]

res = pd.DataFrame.from_dict(
        {f"{param}_{val}": eval_model(XGBClassifier(
            n_estimators=200, 
            learning_rate=0.1,
            reg_alpha=1e-3,
            tree_method="gpu_hist",
            scale_pos_weight=12,
            **{param: val}), df_train)
                                      for val in par_vals}
    )
sns.boxplot(data=res)

In [ ]:
from sklearn.ensemble import StackingClassifier
models = [
    ("rfV1", RandomForestClassifier(n_estimators=200, n_jobs=2)),
    ("rfV2", RandomForestClassifier(n_estimators=200, class_weight="balanced", max_features=1, n_jobs=2)),
    ("xgb", XGBClassifier(n_estimators=5000, 
            learning_rate=0.01,
            tree_method="gpu_hist",
            scale_pos_weight=7.13,
            min_child_weight=2,
            colsample_bytree=0.6)),
]

final_estimator = RandomForestClassifier(n_estimators=1000, class_weight="balanced", n_jobs=2)
model = StackingClassifier(estimators=models, final_estimator=final_estimator, cv=CV, verbose=1000)

df_train = data_preprocess(DROP_COLS, [income_by_state])[0]
eval_model(model, df_train)

In [4]:
df_train, df_test = data_preprocess(
        drop_cols=[],
        single_transforms=SINGLE_TRANSFORMS,
        double_transforms=DOUBLE_TRANSFORMS
    )
df_train.head()

Training data shape: (252000, 17) ; Test data shape: (28000, 16)


,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag,current_job_years_per_experience,age_per_experience,default_per_city,default_per_state,default_per_profession
0,1303835,23,3,1,2,0,33,251,13,3,13,0,1.000000,7.666667,0.121554,0.154369,0.111558
1,7574516,40,10,1,2,0,43,227,14,9,13,0,0.900000,4.000000,0.122497,0.113254,0.148427
2,3991815,66,4,0,2,0,47,8,12,4,10,0,1.000000,16.500000,0.130814,0.167097,0.134167
3,6256451,41,2,1,2,1,43,54,17,2,12,1,1.000000,20.500000,0.326194,0.142550,0.148427
4,5768871,47,11,1,2,0,11,296,22,3,14,1,0.272727,4.272727,0.121137,0.103163,0.115794


In [13]:
y = df_train["current_job_years_per_experience"].values
y[-2]


nan

In [ ]:
from sklearn.model_selection import train_test_split
def get_model_outputs(models):
    df_train = data_preprocess(
            drop_cols=["car_ownership", "house_ownership", "married"],
            transform_fns=[income_by_state]
        )[0]

    scores = {}
    for name, model in models.items():
        print(f"Cross-validating: {name}")
        scores[name] = eval_model(model, df_train)
    scores = pd.DataFrame.from_dict(scores)
    
    X, y = df_train.drop("risk_flag", axis=1).values, df_train["risk_flag"].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
    
    for name in models.keys():
        models[name].fit(X_train, y_train)
    
    preds = {name: model.predict(X_test) for name, model in models.items()}
    return preds, scores